# Inicialização do ambiente

## Importando as bibliotecas

In [ ]:
import sys
import csv
import math
import time
import random
import numpy as np
import pandas as pd
import plotly.express as px
from google.colab import files
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
pip install neupy

     |████████████████████████████████| 226 kB 5.2 MB/s 
     |████████████████████████████████| 92.7 MB 27 kB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.2 MB 59.6 MB/s 
     |████████████████████████████████| 367 kB 75.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
  Attempting uninstall: progressbar2
    Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progres

In [ ]:
import numpy as np
from neupy import algorithms
from neupy.layers import *

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/p

## Conectando com o Drive 

In [ ]:
#para salvar os resultados no Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Instalando biblioteca para tempo de execução

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 221 µs (started: 2021-08-24 01:42:55 +00:00)


# Carregar os dados

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

# google_id = '1DgYTvfbzTe8CPMOwXMupmTOiMTP56PwD' #conj completo
google_id = '1Jsg6ggQF98LT4D2QnEPTjj_XfTdeTWHh' #conj de treinamento
gdd.download_file_from_google_drive(file_id=google_id, 
                                    dest_path = './data.csv', 
                                    showsize = True)
data = pd.read_csv("data.csv")

4.4 MiB Done.
time: 2.31 s (started: 2021-08-24 01:42:55 +00:00)


In [ ]:
data.head()

,Case ID,Activity,Resource,Workload,Duration(s)
0,173961,W_Fixing incoming lead,10889,"{11180: 'HIGH', 10982: 'FREE', 11121: 'FREE', ...",286.0
1,173781,W_Filling in information for the application,11180,"{11180: 'FREE', 10982: 'FREE', 11121: 'FREE', ...",622.0
2,173934,W_Fixing incoming lead,10889,"{11180: 'FREE', 10982: 'FREE', 11121: 'FREE', ...",305.0
3,173901,W_Fixing incoming lead,10889,"{11180: 'HIGH', 10982: 'FREE', 11121: 'FREE', ...",174.0
4,173904,W_Fixing incoming lead,10889,"{11180: 'HIGH', 10982: 'FREE', 11121: 'FREE', ...",351.0


time: 41.9 ms (started: 2021-08-24 01:42:57 +00:00)


In [ ]:
len(data['Case ID'].unique())

2826

time: 10.3 ms (started: 2021-08-24 01:42:57 +00:00)


In [ ]:
len(data['Activity'].unique())

6

time: 11.7 ms (started: 2021-08-24 01:42:57 +00:00)


# Modelagem

## Definição dos elementos e análise de quantidades

- Estado: Activity + Resources Workload

- Ação: Resource

- Custo: Duration

Quantos estados diferentes podemos ter?

\#resources (do notebook de geração do dataset)

x 

\#níveis de worload que as resources podem apresentar (FREE, LOW, HIGH)

x 

\#atividades


In [ ]:
57*3*len(data['Activity'].unique())

1026

time: 8.61 ms (started: 2021-08-24 01:42:57 +00:00)


Portanto, são 1026 estados possíveis no contexto do problema se considerarmos todas as atividades, com todos os níveis de workload para todos os recursos.

In [ ]:
1026 * 57

58482

time: 5.2 ms (started: 2021-08-24 01:42:57 +00:00)


E 58 mil combinações possíveis de todos os estados com todas as ações disponíveis.

In [ ]:
print('#workloads possíveis:',(57-28)*3)
print('#estados posssíveis (workloads x atividades):',(57-27)*3*len(data['Activity'].unique()))
print('#combinações possíveis de estado-ações:',(57-27)*3*len(data['Activity'].unique())*(57-27))

#workloads possíveis: 87
#estados posssíveis (workloads x atividades): 540
#combinações possíveis de estado-ações: 16200
time: 16.7 ms (started: 2021-08-24 01:42:57 +00:00)


## Geração do histórico H 

Formato *State*, *Action*, *Cost*

In [ ]:
H = pd.DataFrame()
H['case'] = data['Case ID']
H['state'] = data['Activity'] + "-" + data['Workload']
H['action'] = data['Resource']
H['cost'] = data['Duration(s)']
H.head()

,case,state,action,cost
0,173961,"W_Fixing incoming lead-{11180: 'HIGH', 10982: ...",10889,286.0
1,173781,W_Filling in information for the application-{...,11180,622.0
2,173934,"W_Fixing incoming lead-{11180: 'FREE', 10982: ...",10889,305.0
3,173901,"W_Fixing incoming lead-{11180: 'HIGH', 10982: ...",10889,174.0
4,173904,"W_Fixing incoming lead-{11180: 'HIGH', 10982: ...",10889,351.0


time: 50 ms (started: 2021-08-24 01:42:57 +00:00)


In [ ]:
H_original = H.copy()

time: 2.48 ms (started: 2021-08-24 01:42:57 +00:00)


## Analisando H

Quantos estados diferentes temos?

In [ ]:
len(H['state'].unique())

1755

time: 17.7 ms (started: 2021-08-24 01:42:57 +00:00)


Quantos recursos diferentes?

In [ ]:
len(H['action'].unique())

30

time: 7.56 ms (started: 2021-08-24 01:42:57 +00:00)


Qual o máximo de combinações state-action podemos ter, então?

In [ ]:
len(H['state'].unique())*len(H['action'].unique())

52650

time: 16.1 ms (started: 2021-08-24 01:42:57 +00:00)


# RL

## Pré-definições e análises

Definição do parâmetro MAX_DIFF - um dos critérios de parada:

In [ ]:
px.box(H, y='cost') # para escolher o MAXDIFF - Q3

time: 1.46 s (started: 2021-08-24 01:42:57 +00:00)


In [ ]:
H.cost.quantile(.25)

62.0

time: 8.4 ms (started: 2021-08-24 01:42:59 +00:00)


Para cada ação (recurso) em cada estado, o próximo estado pode ser diferente (processo estocástico) e mesmo para um mesmo próximo estado, o custo pode ser diferentes. E qual custo usar? Será que há diferença entre o custo médio, mínimo e máximo pra cada tupla estado-ação-próximo_estado?

In [ ]:
#pd.DataFrame(Q.groupby(['state','action','next_state'])['cost'].agg(['mean','max','min'])).round(2)

time: 1.51 ms (started: 2021-08-24 01:42:59 +00:00)


Como podemos ver nos resultados exibidos para as duas primeiras linhas, há diferenças. Vamos usar a média como um primeiro teste.

## Algoritmo

In [ ]:
ALG = 'FQI'
GAMMA = 0.9
MAX_IT = 100
MAX_DIFF = 0
MAX_TIME = 10*60*60 #treinamento de no max 10h
print('MAX_DIFF =', MAX_DIFF)

def insert_next_s(case):
  case['next_state'] = case['state'].shift(-1)
  case = case.fillna('END')
  return case

def get_y(x, n, q, gamma):#, scale):
  q_next_s = 0
  if x['next_state'] != 'END':
    q_next_s = q[q['state']==x['next_state']].groupby('action')[n-1].mean().min()
  #if scale:
  #  scaler = StandardScaler().fit(q_next_s)
  #  return scaler, scaler.transform(x['cost'] + gamma * q_next_s)
  #return None, x['cost'] + gamma * q_next_s
  return x['cost'] + gamma * q_next_s

#def set_q(pred_values, scaler):
#  if scaler is not None:
#    return scaler.inverse_transform(pred_values)
#  return pred_values

def train(model, x_train, q, max_it = MAX_IT, max_diff = MAX_DIFF, gamma = GAMMA):#, scale = False):
  start_time = time.time()
  for n in range(1,max_it):
    print(n)
    y = q.apply(lambda x: get_y(x, n, q, gamma), axis=1)
    model = model.fit(x_train, y)
    #model.fit(x_train, y)
    q[n] = model.predict(x_train)
    if (q[n-1] - q[n]).sum() < max_diff or (time.time()-start_time)>MAX_TIME: 
      break
  return q, n, time.time()-start_time

MAX_DIFF = 0
time: 35.6 ms (started: 2021-08-24 01:42:59 +00:00)


## Inicialização

In [ ]:
def init_q(h):
  aux = h.groupby('case').apply(insert_next_s)
  aux[0] = h.cost.max()
  return aux

time: 3.34 ms (started: 2021-08-24 01:42:59 +00:00)


In [ ]:
h = H.copy()
MAX_DIFF = H.cost.quantile(.25)
q = init_q(h)
q['action'] = q['action'].astype(str) # para que o get_dummies funcione corretamente para essa coluna
#x = pd.get_dummies(q[['state','action']])

time: 6.42 s (started: 2021-08-24 01:42:59 +00:00)


In [ ]:
from ast import literal_eval
x = (q['state'].apply(lambda x: pd.Series(literal_eval(x.split('-')[1])))
               .replace('FREE',0).replace('LOW',1).replace('HIGH',2))
x.columns = ['WL_'+str(col) for col in x.columns]
print(x.shape)
activities_dummies = pd.get_dummies(q['state'].apply(lambda x: x.split('-')[0]))
x = x.join(activities_dummies)
resources_dummies = pd.get_dummies(q['action'])
x = x.join(resources_dummies)
print('#linhas (datapoints), #colunas (features):', x.shape)

(9095, 30)
#linhas (datapoints), #colunas (features): (9095, 66)
time: 6.5 s (started: 2021-08-24 01:43:05 +00:00)


In [ ]:
x.head()

,WL_11180,WL_10982,WL_11121,WL_10609,WL_10899,WL_10629,WL_11049,WL_11201,WL_10889,WL_11119,WL_11179,WL_11169,WL_10809,WL_11122,WL_11181,WL_11009,WL_11189,WL_10138,WL_10881,WL_10909,WL_10972,WL_11203,WL_10913,WL_11000,WL_10861,WL_11259,WL_10932,WL_10910,WL_10929,WL_11003,W_Assessing the application,W_Calling after sent offers,W_Calling to add missing information to the application,W_Evaluate fraud,W_Filling in information for the application,W_Fixing incoming lead,10138,10609,10629,10809,10861,10881,10889,10899,10909,10910,10913,10929,10932,10972,10982,11000,11003,11009,11049,11119,11121,11122,11169,11179,11180,11181,11189,11201,11203,11259
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


time: 92 ms (started: 2021-08-24 01:43:12 +00:00)


In [ ]:
h.cost.max()

239335.0

time: 3.63 ms (started: 2021-08-24 01:43:12 +00:00)


## Execuções

In [ ]:
def save_exec(alg, alg_espec, model_type, model_espec, gamma, it, train_time, q):
  #save metadata
  exec_metadata = [alg, alg_espec, model_type, model_espec, gamma, it, train_time]
  with open(r'/gdrive/My Drive/ppar_results/metadata.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(exec_metadata)
  #save q
  q_filename = ('-').join([str(info) for info in exec_metadata[:-2]])
  with open('/gdrive/My Drive/ppar_results/%s.csv'%q_filename, 'w') as f:
    q.to_csv(f, index=False)

time: 8.33 ms (started: 2021-08-24 01:43:12 +00:00)


#### FQI - LinearRegression

In [ ]:
q, it, train_time = train(LinearRegression(), x, init_q(h))
save_exec(ALG, 'none', 'linear_regression_arrumado', 'default', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
time: 7min 29s (started: 2021-08-13 18:34:20 +00:00)


#### FQI - Random Forest

In [ ]:
q, it, train_time = train(RandomForestRegressor(n_estimators=5), x, init_q(h))
save_exec(ALG, 'none', 'random_forest', 'n_estimators=5', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
time: 3min 14s (started: 2021-08-24 01:43:36 +00:00)


In [ ]:
q, it, train_time = train(RandomForestRegressor(n_estimators=10), x, init_q(h))
save_exec(ALG, 'none', 'random_forest', 'n_estimators=10', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
time: 3min 2s (started: 2021-08-24 01:46:51 +00:00)


In [ ]:
q, it, train_time = train(RandomForestRegressor(n_estimators=20), x, init_q(h))
save_exec(ALG, 'none', 'random_forest', 'n_estimators=20', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
time: 3min 56s (started: 2021-08-24 01:49:53 +00:00)


In [ ]:
q, it, train_time = train(RandomForestRegressor(n_estimators=50), x, init_q(h))
save_exec(ALG, 'none', 'random_forest', 'n_estimators=20', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
time: 5min 23s (started: 2021-08-24 01:53:50 +00:00)


In [ ]:
q, it, train_time = train(RandomForestRegressor(), x, init_q(h))
save_exec(ALG, 'none', 'random_forest', 'default', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
time: 4min 57s (started: 2021-08-24 01:59:46 +00:00)


#### NFQ - MLP

In [ ]:
q, it, train_time = train(MLPRegressor(learning_rate='adaptive', alpha=0.005, hidden_layer_sizes=(10,10)), x, init_q(h))
save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, alpha=0.005, hidden_layer_sizes=(10,10)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

21


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



22


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 7min 50s (started: 2021-08-24 02:04:49 +00:00)


In [ ]:
q, it, train_time = train(MLPRegressor(learning_rate='adaptive', alpha=0.005, hidden_layer_sizes=(50,50)), x, init_q(h))
save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, alpha=0.005, hidden_layer_sizes=(50,50)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3
4


...

25


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



26


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 14min 12s (started: 2021-08-24 02:12:40 +00:00)


In [ ]:
# Mesmo com max_iter=1000, nao converge
q, it, train_time = train(MLPRegressor(learning_rate='adaptive', max_iter = 1000), x, init_q(h))
save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



3


...

23


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



24


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.



time: 31min 46s (started: 2021-08-24 02:26:53 +00:00)


In [ ]:
q, it, train_time = train(MLPRegressor(learning_rate = 'adaptive', activation = 'logistic'), x, init_q(h))
save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, activation=logistic', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

6


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



7


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 4min 13s (started: 2021-08-24 02:58:39 +00:00)


In [ ]:
q, it, train_time = train(MLPRegressor(learning_rate = 'adaptive', activation = 'logistic', hidden_layer_sizes=(50,50)), x, init_q(h))
save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, activation=logistic, hidden_layers=(50,50)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

4


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



5


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 3min 12s (started: 2021-08-24 03:02:53 +00:00)


#### NFQ - RProp

In [ ]:
#Pra comparar com as primeiras execucoes de MLP, que usa Relu por padrao
network = Input(x.shape[1]) >> Relu(50) >> Relu(50) >> Relu(1)
q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
save_exec(ALG, 'none', 'RPROP', '50,50_relu', GAMMA, it, train_time, q)

Instructions for updating:
Colocations handled automatically by placer.
1


2


3


4


5


...

10


11


12


13


time: 4min 54s (started: 2021-08-24 03:06:06 +00:00)


In [ ]:
#network = Input(x.shape[1]) >> Sigmoid(500) >> Sigmoid(500) >> Sigmoid(1)
q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
save_exec(ALG, 'none', 'RPROP', '500,500-Sigmoid', GAMMA, it, train_time, q)

1


2


3


4


5


...

9


10


11


12


time: 4min 5s (started: 2021-08-24 03:11:00 +00:00)


In [ ]:
#network = Input(x.shape[1]) >> Sigmoid(100) >> Sigmoid(1)
q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
save_exec(ALG, 'none', 'RPROP', '100-Sigmoid', GAMMA, it, train_time, q)

1


2


3


4


5


...

9


10


11


12


time: 3min 46s (started: 2021-08-24 03:15:06 +00:00)
